<a href="https://colab.research.google.com/github/SwetaSengupta/DS-Unit-4-Sprint-2-Neural-Networks/blob/main/LS_DS_424_Hyperparameter_Tuning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [ ]:
##### Your Code Here #####

In [ ]:
import pandas as pd
df = pd.read_csv("https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv")
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [ ]:
!pip install category_encoders==2.*
!pip install pandas-profiling==2.*

     |████████████████████████████████| 81kB 2.2MB/s 
     |████████████████████████████████| 266kB 2.8MB/s 
     |████████████████████████████████| 3.1MB 8.9MB/s 
     |████████████████████████████████| 604kB 23.6MB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
     |████████████████████████████████| 71kB 6.9MB/s 
     |████████████████████████████████| 71kB 8.7MB/s 
     |████████████████████████████████| 296kB 28.8MB/s 
  Created wheel for htmlmin: filename=htmlmin-0.1.12-cp36-none-any.whl size=27084 sha256=8398a03b0d0a63c11ad8ab3c2503823a89c46e92d23adb360f667ee63c188384
  Stored in directory: /root/.cache/pip/wheels/43/07/ac/7c5a9d708d65247ac1f94066cf1db075540b85716c30255459
  Created wheel for imagehash: filename=ImageHash-4.1.0-py2.py3-none-any.whl size=291990 sha256=5a74efd815e48a2bd3cb83d2ef7c719dd04c13a38417c9645746beba20a0caf0
  Stored in directory: /root/.cache/pip/wheels/07/1c/dc/6831446f09feb8cc199ec73a0f2f0703253f6ae013a22f4be9
Successfully built htmlmin imageh

Load and explore the data

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df,train_size=.80,test_size=.2,
                               stratify=df['Churn'],random_state=7)
train.shape, test.shape

((5634, 21), (1409, 21))

In [ ]:
# get a profile
import pandas_profiling
from pandas_profiling import ProfileReport
profile = ProfileReport(train,minimal=True)

profile.to_notebook_iframe()
profile.to_file(output_file='customer_churn_data_profile.html')

Cleaning and encoding the data

In [ ]:
def wrangle(X):
  X = X.copy()

  # drop columns that are all unique
  X = X.drop(columns=["customerID", "TotalCharges", "tenure", "MonthlyCharges"])

  return X

In [ ]:
# wrangle the test and train sets
w_train = wrangle(train)
w_test = wrangle(test)
print(train.shape, w_train.shape)
print(test.shape, w_test.shape)

(5634, 21) (5634, 17)
(1409, 21) (1409, 17)


In [ ]:
from sklearn.preprocessing import OrdinalEncoder

def ordinal_encode(train, test):
  
  features = train.columns.tolist()

  enc = OrdinalEncoder()
  enc.fit(train)
  train = pd.DataFrame(enc.transform(train),columns=features)
  test = pd.DataFrame(enc.transform(test),columns=features)

  return train, test

In [ ]:
# encode the data
o_train, o_test = ordinal_encode(w_train, w_test)

In [ ]:
# separate into X and y
target = 'Churn'
features = o_train.drop(columns=target).columns.tolist()
X_train = o_train[features]
y_train = o_train[target]
X_test = o_test[features]
y_test = o_test[target]
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(5634, 16) (5634,) (1409, 16) (1409,)


In [ ]:
# check that it worked
X_train.head()

,gender,SeniorCitizen,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod
0,1.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,2.0,2.0,2.0,1.0,1.0,2.0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,2.0,0.0,1.0,2.0
2,1.0,0.0,0.0,0.0,1.0,0.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,2.0
4,1.0,0.0,1.0,1.0,1.0,2.0,1.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,0.0


Creating a baseline

In [ ]:
y_train.value_counts(normalize = True)

0.0    0.734647
1.0    0.265353
Name: Churn, dtype: float64

Scaling the data

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Hyperparametertuning

In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Important Hyperparameters
X =  X_train
y =  y_train

inputs = X.shape[1]
epochs = 20
batch_size = 8

# Create Model
model = Sequential()
model.add(Dense(64, activation='sigmoid', input_shape=(inputs,)))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))

# Early Stopping
callback=EarlyStopping(patience=3,verbose=1,min_delta=.001)

# Compile Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit Model
results = model.fit(X_train, y_train, 
            validation_data=(X_test,y_test), 
            epochs=epochs, 
            batch_size=batch_size,
            callbacks=[callback])

Epoch 1/20
705/705 [==============================] - 2s 2ms/step - loss: 0.4862 - accuracy: 0.7558 - val_loss: 0.4470 - val_accuracy: 0.7786
Epoch 2/20
705/705 [==============================] - 1s 2ms/step - loss: 0.4530 - accuracy: 0.7723 - val_loss: 0.4403 - val_accuracy: 0.7828
Epoch 3/20
705/705 [==============================] - 1s 2ms/step - loss: 0.4509 - accuracy: 0.7767 - val_loss: 0.4380 - val_accuracy: 0.7871
Epoch 4/20
705/705 [==============================] - 1s 2ms/step - loss: 0.4498 - accuracy: 0.7742 - val_loss: 0.4373 - val_accuracy: 0.7793
Epoch 5/20
705/705 [==============================] - 1s 2ms/step - loss: 0.4521 - accuracy: 0.7746 - val_loss: 0.4378 - val_accuracy: 0.7793
Epoch 6/20
705/705 [==============================] - 1s 2ms/step - loss: 0.4493 - accuracy: 0.7765 - val_loss: 0.4369 - val_accuracy: 0.7807
Epoch 7/20
705/705 [==============================] - 1s 2ms/step - loss: 0.4487 - accuracy: 0.7776 - val_loss: 0.4418 - val_accuracy: 0.7757
Epoch 

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                1088      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 3,201
Trainable params: 3,201
Non-trainable params: 0
_________________________________________________________________


In [ ]:
scores = model.evaluate(X_test, y_test)
print(f'{model.metrics_names[1]}: {scores[1]*100}%')

45/45 [==============================] - 0s 969us/step - loss: 0.4396 - accuracy: 0.7764
accuracy: 77.64371633529663%


Hyperparametertuning using  Grid Search and Cross Validation

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dropout

In [ ]:
# testing batch size 
def create_model():
  # create model
  model = Sequential()
  model.add(Dense(16, input_dim = 16, activation='relu'))
  model.add(Dense(16, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  # Compile Model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [20]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7795522809028625 using {'batch_size': 40, 'epochs': 20}
Means: 0.770677900314331, Stdev: 0.0063182461757646715 with: {'batch_size': 10, 'epochs': 20}
Means: 0.779021155834198, Stdev: 0.012136708095816106 with: {'batch_size': 20, 'epochs': 20}
Means: 0.7795522809028625, Stdev: 0.007900565040947046 with: {'batch_size': 40, 'epochs': 20}
Means: 0.7788425922393799, Stdev: 0.008201692691327436 with: {'batch_size': 60, 'epochs': 20}
Means: 0.771920132637024, Stdev: 0.012441121549128697 with: {'batch_size': 80, 'epochs': 20}
Means: 0.7747617363929749, Stdev: 0.009398874132611108 with: {'batch_size': 100, 'epochs': 20}


In [ ]:
# Optimizer

def create_model(optimizer='adam'):
    model = Sequential()
    model.add(Dense(16, input_dim=16, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
          
    model.compile(loss = "binary_crossentropy", optimizer = optimizer, metrics=['accuracy'])
    return model

#creating model
model = KerasClassifier(build_fn=create_model, epochs=20, batch_size=100, verbose=0)

#define the grid search parameters
param_grid = {'optimizer': ['adam', 'nadam', 'sgd', 'adadelta', 'adagrad']}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7736958742141724 using {'optimizer': 'adam'}
Means: 0.7736958742141724, Stdev: 0.012286612322051593 with: {'optimizer': 'adam'}
Means: 0.7728079319000244, Stdev: 0.010648039762823396 with: {'optimizer': 'nadam'}
Means: 0.746892762184143, Stdev: 0.015564053567740913 with: {'optimizer': 'sgd'}
Means: 0.556459492444992, Stdev: 0.11530364650651237 with: {'optimizer': 'adadelta'}
Means: 0.739082682132721, Stdev: 0.013774203942308565 with: {'optimizer': 'adagrad'}


In [ ]:
#learning rate
def create_model(learning_rate):
  # create model
  model = Sequential(name="nn1")
  model.add(Dense(16, input_dim = 16, activation='relu', name='In'))
  model.add(Dense(16, activation='relu', name='1'))
  model.add(Dense(16, activation='relu', name='2'))       
  model.add(Dense(1, activation='sigmoid', name='out'))
  sgd = SGD(lr=learning_rate, momentum=0.0, nesterov=False) 
  # Compile Model
  model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
  return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
param_grid = {'learning_rate': [.001, .01, .1, .2, .3, .5]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7678430914878845 using {'learning_rate': 0.3}
Means: 0.6042113482952118, Stdev: 0.14474282637853225 with: {'learning_rate': 0.001}
Means: 0.7351794481277466, Stdev: 0.0049326460974478984 with: {'learning_rate': 0.01}
Means: 0.7564796686172486, Stdev: 0.005691753457149616 with: {'learning_rate': 0.1}
Means: 0.7671269178390503, Stdev: 0.011438350740736663 with: {'learning_rate': 0.2}
Means: 0.7678430914878845, Stdev: 0.02131145628513375 with: {'learning_rate': 0.3}
Means: 0.7667745351791382, Stdev: 0.016259221193278873 with: {'learning_rate': 0.5}


In [ ]:
# Network Weight Initialization 

def create_model(init_mode='uniform'):
    model = Sequential()
    model.add(Dense(16, input_dim=16, kernel_initializer=init_mode, activation='relu'))
    model.add(Dense(8, kernel_initializer=init_mode, activation='relu'))
    model.add(Dense(1, kernel_initializer=init_mode, activation='sigmoid'))
          
    model.compile(loss = "binary_crossentropy", optimizer ='adam', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, epochs=20, batch_size=100, verbose=0)

param_grid = {'init_mode': ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7836351871490479 using {'init_mode': 'normal'}
Means: 0.7772444725036621, Stdev: 0.007959290396090439 with: {'init_mode': 'uniform'}
Means: 0.7774246215820313, Stdev: 0.008340978799966177 with: {'init_mode': 'lecun_uniform'}
Means: 0.7836351871490479, Stdev: 0.011761420877558092 with: {'init_mode': 'normal'}
Means: 0.7346473693847656, Stdev: 0.0038052983546973205 with: {'init_mode': 'zero'}
Means: 0.7770695567131043, Stdev: 0.011569537369930159 with: {'init_mode': 'glorot_normal'}
Means: 0.7751163601875305, Stdev: 0.009410779746545582 with: {'init_mode': 'glorot_uniform'}
Means: 0.7669540286064148, Stdev: 0.013040698095272679 with: {'init_mode': 'he_normal'}
Means: 0.7712098121643066, Stdev: 0.010013638172351534 with: {'init_mode': 'he_uniform'}


In [ ]:
# Dropout Regularization 

from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Dropout
from tensorflow.keras.constraints import MaxNorm

def create_model(dropout_rate=0.0, weight_constraint=0):
    model = Sequential()
    model.add(Dense(16, input_dim=16, kernel_initializer='normal', activation='relu'))
    model.add(Dense(8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
          
    model.compile(loss = "binary_crossentropy", optimizer ='adam', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, epochs=20, batch_size=100, verbose=0)

param_grid = {'weight_constraint': [1, 2, 3, 4, 5],
              'dropout_rate': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.784346604347229 using {'dropout_rate': 0.5, 'weight_constraint': 5}
Means: 0.7784895658493042, Stdev: 0.011584992216116665 with: {'dropout_rate': 0.0, 'weight_constraint': 1}
Means: 0.7813289523124695, Stdev: 0.008636449627976526 with: {'dropout_rate': 0.0, 'weight_constraint': 2}
Means: 0.7809722900390625, Stdev: 0.0058832395971009956 with: {'dropout_rate': 0.0, 'weight_constraint': 3}
Means: 0.7752922296524047, Stdev: 0.010842456495206437 with: {'dropout_rate': 0.0, 'weight_constraint': 4}
Means: 0.7799095749855042, Stdev: 0.011370089495460353 with: {'dropout_rate': 0.0, 'weight_constraint': 5}
Means: 0.7823937177658081, Stdev: 0.011554093596829662 with: {'dropout_rate': 0.1, 'weight_constraint': 1}
Means: 0.7756482720375061, Stdev: 0.011901870869925932 with: {'dropout_rate': 0.1, 'weight_constraint': 2}
Means: 0.7806183218955993, Stdev: 0.010536428797045652 with: {'dropout_rate': 0.1, 'weight_constraint': 3}
Means: 0.7822165846824646, Stdev: 0.012768001775259198 with: {'drop

In [ ]:
# Neurons in Hidden Layer

def create_model(neurons=1):
    model = Sequential()
    model.add(Dense(neurons, input_dim=16, kernel_initializer='normal', kernel_constraint=MaxNorm(3), activation='relu'))
    model.add(Dense(8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
          
    model.compile(loss = "binary_crossentropy", optimizer ='adam', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, epochs=20, batch_size=100, verbose=0)

param_grid = {'neurons': [1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7841693043708802 using {'neurons': 30}
Means: 0.7346473693847656, Stdev: 0.0038052983546973205 with: {'neurons': 1}
Means: 0.7696074485778809, Stdev: 0.018611568410931825 with: {'neurons': 5}
Means: 0.7765366792678833, Stdev: 0.011188845337839783 with: {'neurons': 10}
Means: 0.7781354069709778, Stdev: 0.012790827991430002 with: {'neurons': 15}
Means: 0.7816843509674072, Stdev: 0.008579370234173003 with: {'neurons': 20}
Means: 0.7786664009094239, Stdev: 0.008132885567011333 with: {'neurons': 25}
Means: 0.7841693043708802, Stdev: 0.010984173549594354 with: {'neurons': 30}
Means: 0.7809741973876954, Stdev: 0.01142332306191395 with: {'neurons': 35}
Means: 0.7809738636016845, Stdev: 0.008671402063245943 with: {'neurons': 40}
Means: 0.7790222525596618, Stdev: 0.011573109823520649 with: {'neurons': 45}
Means: 0.7820394277572632, Stdev: 0.0067856124471376905 with: {'neurons': 50}


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?